In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score

In [2]:
import pandas as pd

# Cargar el conjunto de datos desde un archivo CSV
data_path = r"C:\Proyectos programación\Modelos\consumo_casa.csv"
data = pd.read_csv(data_path)

# Eliminar la primera columna (asumiendo que se llama 'Unnamed: 0')
data.drop(columns=['Unnamed: 0'], inplace=True)

# Convertir la columna 'Fecha' a tipo datetime si no lo está
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Dividir la columna 'Fecha' en mes, día, hora y minuto
data['Mes'] = data['Fecha'].dt.month
data['Dia'] = data['Fecha'].dt.day
data['Hora'] = data['Fecha'].dt.hour
data['Minuto'] = data['Fecha'].dt.minute

# Agregar una nueva columna para el día de la semana en formato de texto
data['Dia_semana'] = data['Fecha'].dt.day_name()

# Convertir la columna 'Dia_semana' en variables dummy
data = pd.get_dummies(data, columns=['Dia_semana'])

data.drop(columns=['Fecha'], inplace=True)

print(data.columns)


Index(['Medidor [W]', 'Refrigerator', 'Clothes washer', 'Clothes Iron',
       'Computer', 'Oven', 'Play', 'TV', 'Sound system', 'Mes', 'Dia', 'Hora',
       'Minuto', 'Dia_semana_Friday', 'Dia_semana_Monday',
       'Dia_semana_Saturday', 'Dia_semana_Sunday', 'Dia_semana_Thursday',
       'Dia_semana_Tuesday', 'Dia_semana_Wednesday'],
      dtype='object')


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

# Aplicar función seno a las variables de predicción (y)
variables_prediccion = ['Refrigerator', 'Clothes washer', 'Clothes Iron', 'Computer', 'Oven', 'Play', 'TV', 'Sound system', 'Medidor [W]']
for variable in variables_prediccion:
    data[f'{variable}_seno'] = np.sin(2 * np.pi * data[variable] / 24)

# Dividir los datos en características (X) y etiquetas (y)
X = data[['Mes', 'Dia', 'Hora', 'Minuto', 'Medidor [W]_seno','Dia_semana_Friday', 'Dia_semana_Monday',
       'Dia_semana_Saturday', 'Dia_semana_Sunday', 'Dia_semana_Thursday',
       'Dia_semana_Tuesday', 'Dia_semana_Wednesday']]
y = data[[f'{variable}_seno' for variable in variables_prediccion if variable != 'Medidor [W]']]

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# Construir y entrenar el modelo de red neuronal
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(64, activation=lambda x: tf.math.sin(x)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation=lambda x: tf.math.cos(x)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='linear'))
model.compile(optimizer='adam', loss='mse')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluar el modelo
train_score = model.evaluate(X_train, y_train, verbose=0)
test_score = model.evaluate(X_test, y_test, verbose=0)

print("MSE en entrenamiento:", train_score)
print("MSE en prueba:", test_score)


Epoch 1/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.2760 - val_loss: 0.0823
Epoch 2/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 0.0831 - val_loss: 0.0748
Epoch 3/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0771 - val_loss: 0.0651
Epoch 4/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 0.0712 - val_loss: 0.0629
Epoch 5/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.0694 - val_loss: 0.0619
Epoch 6/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - loss: 0.0682 - val_loss: 0.0614
Epoch 7/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 0.0683 - val_loss: 0.0611
Epoch 8/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - loss: 0.0672 - val_loss: 0.0608
Epoch 9/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 0.0675 - val_loss: 0.0608
Epoch 10/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 0.0672 - val_loss: 0.0606
Epoch 11/50
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 0.0666 - val_loss: 0.0604
Epoch 12/50
4854/48

In [4]:
print(X.columns)

Index(['Mes', 'Dia', 'Hora', 'Minuto', 'Medidor [W]_seno', 'Dia_semana_Friday',
       'Dia_semana_Monday', 'Dia_semana_Saturday', 'Dia_semana_Sunday',
       'Dia_semana_Thursday', 'Dia_semana_Tuesday', 'Dia_semana_Wednesday'],
      dtype='object')


In [5]:
from sklearn.metrics import mean_squared_error

# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)

print("Error cuadrático medio:", mse)


1214/1214 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Error cuadrático medio: 0.058261532


In [6]:
from sklearn.metrics import mean_squared_error

# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio por cada columna
mse_por_columna = {}
for i, columna in enumerate(y.columns):
    mse_por_columna[columna] = mean_squared_error(y_test.iloc[:, i], y_pred[:, i])

# Imprimir el MSE por cada columna
for columna, mse_valor in mse_por_columna.items():
    print(f"MSE para {columna}: {mse_valor}")


1214/1214 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
MSE para Refrigerator_seno: 0.18617171049118042
MSE para Clothes washer_seno: 0.03733524680137634
MSE para Clothes Iron_seno: 0.014297968707978725
MSE para Computer_seno: 0.030152691528201103
MSE para Oven_seno: 0.016584323719143867
MSE para Play_seno: 0.05992512032389641
MSE para TV_seno: 0.0994337871670723
MSE para Sound system_seno: 0.022193606942892075


In [7]:
# Elegir la variable para la cual mostrar los valores (por ejemplo, 'Refrigerator')
variable_elegida = 'Play'

# Obtener los primeros 5 valores reales y predichos para la variable elegida
primeros_5_valores = pd.DataFrame({
    'Valor Real': y_test[f'{variable_elegida}_seno'].iloc[:10].values,
    'Valor Predicho': y_pred[:10, y.columns.get_loc(f'{variable_elegida}_seno')]
})

# Mostrar los primeros 5 valores reales y predichos
print(f"Valores reales y predichos para la variable '{variable_elegida}' para los primeros 5 datos:")
print(primeros_5_valores)


Valores reales y predichos para la variable 'Play' para los primeros 5 datos:
   Valor Real  Valor Predicho
0    0.284015        0.181415
1    0.078459        0.238422
2    0.000000        0.409804
3    0.078459        0.099182
4    0.000000        0.046530
5    0.309017        0.134452
6    0.078459        0.139115
7    0.000000        0.562360
8    0.078459        0.214929
9    0.000000        0.069441
